# Analysis of Nanopore eDNA

### Optional background and setup

This notebook assumes the following software has been installed; you don't need to worry about this section, it's a record for the next time we have do this!

- [Fastp](https://github.com/OpenGene/fastp)
    - Use: Allows us to filter DNA reads based on the quality of the data, size, or other factors. 
    - Installation with [conda](https://docs.conda.io/en/latest/miniconda.html): `conda create -y --name fastp fastp==0.20.0 -c bioconda`
- [Porechop ABI](https://github.com/bonsai-team/Porechop_ABI)
    - Use: Allows us to remove adapters or barcode sequences
    - Installation with conda: `conda create -y --name porechop_abi porechop_abi==0.5.0 -c bioconda -c conda-forge)`
- [Amplicon sorter](https://github.com/avierstr/amplicon_sorter)
    - Use: Allows us to group different DNA amplicon sequences (i.e., sequences amplified by PCR) into groups 
    - Installation with conda: 
        - `https://github.com/avierstr/amplicon_sorter`
        - `conda activate amplicon_sorter`
        - `git clone https://github.com/avierstr/amplicon_sorter.git`
        - `cd amplicon_sorter`
        - `python3 -m pip install edlib`
        - `pip3 install biopython`
        - `pip3 install matplotlib`
- [BLAST](http://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastDocs)
    - Use: Allows us to search for species matches 
    - Installation using conda: `conda create -y --name blast blast==2.13.0-0 -c bioconda -c conda-forge`
        
To use BLAST we also need to make a database of sequences to search

*Download sequences from NCBI* 

- [Entrez-search](entrez-direct)
    - Use: Allows us to use command line to download sequences
    - Installation using conda: `conda create -y --name edirect entrez-direct -c bioconda -c conda-forge`

*Search for plants with rbcL or matk sequence and download into a file*

- using taxonomy search https://www.ncbi.nlm.nih.gov/nuccore 
- `esearch -db nucleotide -query "((((txid33090[Organism:exp])) AND rbcl[Gene Name])) OR maturase K "| efetch -format fasta > plant_query_sequences.fasta`
- Check the number of records retrieved: `grep ">"  plant_query_sequences.fasta|wc  -l` # result 500132
- Make a blast database: `makeblastdb -in plant_query_sequences.fasta -dbtype nucl -out plant_db`
- Get the taxonomy database (must be in the formed plant_db folder)
- `update_blastdb.pl taxdb` # must be in blast conda environment

**Sample Processing** 

- Nanopore reads are generally in individual files in groups of 4,000 reads each. All reads for a barcode were combined using cat
   -  `cat *.fastq.gz > combined_edna.fastq.gz`

## Analysis 

In this lab we will analyze the Nanopore sequence data using the following steps. 


1. We will check the quality of the data and filter out low-quality data using [Fastp](https://github.com/OpenGene/fastp)

2. We will remove "adapter" and barcode DNA (extra DNA attached to our reads) using [Porechop ABI](https://github.com/bonsai-team/Porechop_ABI)

3. We will sort similar sequences (likely from the same species) using [Amplicon sorter](https://github.com/avierstr/amplicon_sorter)

4. We will use [BLAST](http://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastDocs) to search for matches in our DNA to known samples 

## Locating the Data

We have placed DNA from this experiment  in the following locations (you can see them when you use the navigation pane on the left)

- **Sample 1**
    `e_dna_data/combined_samples/sample_1.fastq.gz`

- **Sample 2**
    `e_dna_data/combined_samples/sample_2.fastq.gz`

- **Sample 3**
    `e_dna_data/combined_samples/sample_3.fastq.gz`
    
We also have some test data to learn and play with 
- **Test sample**
    `e_dna_data/combined_samples/test_sample.fastq.gz`
 

## Creating a folder for analysis results

We can create a folder using this command below; we will organize and save our results there

In [ ]:
mkdir ~/my_edna_analysis

## Checking data quality



fastp is "A tool designed to provide fast all-in-one preprocessing for FastQ files. This tool is developed in C++ with multithreading supported to afford high performance."

In other words, we can take the FASTQ files (our sequencing data) and do various things to improve the quality of that data such as

- Remove reads that have an overall low quality (e.g., low Phred score)
- Trim portions of a read (e.g., where the first few/last few bases are of low quality)
- Remove reads that are shorter than some desired length (e.g., drop short reads and keep long reads)
- Remove adapters (e.g., in sequencing, some artificial DNA "tags" may be added to the DNA to be sequenced)
- And more...



We activate the software using this command

In [ ]:
conda activate fastp

In the `~/my_edna_analysis` folder we can make a `fastp_analysis` folder to save our results

In [ ]:
mkdir ~/my_edna_analysis/fastp_analysis

We will analyze the test dataset and create a folder for the output results

In [ ]:
mkdir ~/my_edna_analysis/fastp_analysis/test_sample

We can use the following command to examine the data quality. We will run this on the test data. You can then alter the command for your sample

We will use [Fastp](https://github.com/OpenGene/fastp) to get a report on our data and filter out reads that are too big or too small to be our PCR products (we expected ~600bp for rbcL and ~900bp for matk + the adapter sequences which may add some length)

Every setting of the software (called an [argument](https://www.javatpoint.com/linux-arguments)) starts with "--" so:

- `--in1`: Location (and name) of the input file to analyze (expressed as a path, for example `folder1/folder2/file.txt`)
- `--out1`: Location (and name) of the file that should be output 
- `--length_required #`: Minimum length of DNA sequences to keep (in basepairs) followed some integer
- `--length_limit #`: Maximum length of DNA sequences to keep (in basepairs) followed some integer
- `--qualified_quality_phred ##`: Minimum quality average ([phred score](https://en.wikipedia.org/wiki/Phred_quality_score))
- `--thread XX`: How many CPUs to use 
- `--report_title "your title goes here"`: A title to give the report (name as you like)
-  `--html path/to/output.html`: Location (and name) of an HTML (website) report that will be produced
-  `--json path/to/output.json`: Location (and name) of a json (we won't use this) report that will be produced

In the command below, a comment (starts with a "#" ) reminds you of values you can change

In [ ]:
fastp --in1 e_dna_data/combined_samples/test_sample.fastq.gz\
      --out1 my_edna_analysis/fastp_analysis/test_sample/filtered_test_sample.fastq.gz\
      --length_required 600\
      --length_limit 1100\
      --qualified_quality_phred 15\
      --thread 16\
      --report_title "Combined Q15+ reads > 600bp < 1100bp"\
      --html my_edna_analysis/fastp_analysis/test_sample/filtered_test_sample.html\
      --json my_edna_analysis/fastp_analysis/test_sample/filtered_test_sample.json

Running the above will generate a file called `filtered_test_sample.fastq.gz` which you can navigate to in your left navigation pane at `my_edna_analysis > fastp_analysis > test_sample`

### Running fastp on sample 1

To do this analysis on sample 1 you would

Create a folder in `mkdir ~/my_edna_analysis/fastp_analysis` for sample 1

In [ ]:
mkdir ~/my_edna_analysis/fastp_analysis/sample_1

Modify the `fastp` command to use paths and filenames appropriate for that sample 

In [ ]:
fastp --in1 e_dna_data/combined_samples/sample_1.fastq.gz\
      --out1 my_edna_analysis/fastp_analysis/sample_1/filtered_sample_1.fastq.gz\
      --length_required 600\
      --length_limit 1100\
      --qualified_quality_phred 15\
      --thread 16\
      --report_title "Combined Q15+ reads > 600bp < 1100bp"\
      --html my_edna_analysis/fastp_analysis/sample_1/filtered_sample_1.html\
      --json my_edna_analysis/fastp_analysis/sample_1/filtered_sample_1.json

### Things to pay attention to

The value you choose for phred quality score (`--qualified_quality_phred `) and length (`--length_required 600`,`--length_limit 1100`)can have a dramatic effect on how many reads pass the filter. Keep track to the report from `fastp` that notes how many reads passed the filter. After you complete BLAST searches using your initial settings, you could come back to see what happens if early on you become more or less stringent. 

### Running fastp on sample 2 and 3

Using what you have done above, repeat the analysis for sample 2 and 3

Create a folder in `~/my_edna_analysis/fastp_analysis` for sample 2

In [ ]:
mkdir

Create a folder in `~/my_edna_analysis/fastp_analysis` for sample 3

In [ ]:
mkdir

Modify the `fastp` command to use paths and filenames appropriate for sample 2

In [ ]:
fastp --in1 e_dna_data/combined_samples/XXXXXX.fastq.gz\
      --out1 my_edna_analysis/fastp_analysis/sample_XXXXXX/filtered_XXXXXX.fastq.gz\
      --length_required 600\
      --length_limit 1100\
      --qualified_quality_phred 15\
      --thread 16\
      --report_title "Combined Q15+ reads > 600bp < 1100bp"\
      --html my_edna_analysis/fastp_analysis/sample_1/filtered_XXXXXX.html\
      --json my_edna_analysis/fastp_analysis/sample_1/filtered_XXXXXX.json

Modify the `fastp` command to use paths and filenames appropriate for sample 3

In [ ]:
fastp --in1 e_dna_data/combined_samples/XXXXXX.fastq.gz\
      --out1 my_edna_analysis/fastp_analysis/sample_XXXXXX/filtered_XXXXXX.fastq.gz\
      --length_required 600\
      --length_limit 1100\
      --qualified_quality_phred 15\
      --thread 16\
      --report_title "Combined Q15+ reads > 600bp < 1100bp"\
      --html my_edna_analysis/fastp_analysis/sample_1/filtered_XXXXXX.html\
      --json my_edna_analysis/fastp_analysis/sample_1/filtered_XXXXXX.json

## Removing adapter sequences 

We need to remove any barcodes or adapter DNA sequences that were ligated to our PCR products. We will do this using [Porechop ABI](https://github.com/bonsai-team/Porechop_ABI). This time we will use the **filtered** DNA sequences we generated with `fastp`.

Create a folder `~/my_edna_analysis` for data that is processed with `porechopabi` and a folder inside the `porechopabi` named for each sample. We can do this in one command

In [ ]:
mkdir -p my_edna_analysis/porechop_abi_analysis/test_sample

Next activate the `porechopabi` software (we only need to activate software once)

In [ ]:
conda activate porechop_abi

Finally we use the command `porechop_abi`

- `--ab_initio`: Asks the software to determine the adapter sequences for itself
- `-i`: Location (and name) of the input file to analyze (expressed as a path, for example folder1/folder2/file.txt)
- `-o`: Location (and name) of the output file to write (expressed as a path, for example folder1/folder2/file.txt)

Notice that for the test data, we specify to use the filtered data generated by `fastp` and we give a file name for our output the reflects that data has been "chopped" of adapters. 

In [ ]:
porechop_abi --ab_initio \
 -i my_edna_analysis/fastp_analysis/test_sample/filtered_test_sample.fastq.gz\
 -o my_edna_analysis/porechop_abi_analysis/test_sample/chop_filtered_test_sample.fastq.gz

We can find our cleaned dataset in the navigation pane at my_edna_analysis > porechop_abi_analysis > test_sample > chop_filtered_test_sample.fastq.gz

For our next step we need to decompress the file created so we will do that now using the output file. This is a long command but it's basically:

`gzip -d compressed_file.gz uncompressed_file`

After `gzip -d` (the command to decompress) you have the path and name of the compressed file followed by the name and path of where the uncompressed output should be placed. 

In [ ]:
gzip -d my_edna_analysis/porechop_abi_analysis/test_sample/chop_filtered_test_sample.fastq.gz my_edna_analysis/porechop_abi_analysis/test_sample/chop_filtered_test_sample.fastq

### Running porechop_abi on sample 1, 2, and 3

Create output folders for sample 1, 2, and 3

**Hint**: Complete this command `mkdir -p my_edna_analysis/porechop_abi_analysis/YOUR-FOLDER-NAME-HERE` (replace your folder name)

In [ ]:
mkdir -p

In [ ]:
mkdir -p 

In [ ]:
mkdir -p 

Run porechop_abi on each sample (modifying the paths and names of your inputs and outputs)

In [ ]:
porechop_abi --ab_initio \
 -i my_edna_analysis/fastp_analysis/XXXXX/filtered_XXXXX.fastq.gz\
 -o my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq.gz

In [ ]:
porechop_abi --ab_initio \
 -i my_edna_analysis/fastp_analysis/XXXXX/filtered_XXXXX.fastq.gz\
 -o my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq.gz

In [ ]:
porechop_abi --ab_initio \
 -i my_edna_analysis/fastp_analysis/XXXXX/filtered_XXXXX.fastq.gz\
 -o my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq.gz

Decompress the porechop outputs for the next step 

In [ ]:
gzip -d my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq.gz my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq

In [ ]:
gzip -d my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq.gz my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq

In [ ]:
gzip -d my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq.gz my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq

## Running ampliconsorter

Next, we will take our DNA and sort similar reads into "bins" which may represent different species represented in the sample

In [ ]:
mkdir -p my_edna_analysis/amplicon_sorter_analysis/test_sample

Next activate the `amplicon_sorter` software (we only need to activate software once)

In [ ]:
conda activate amplicon_sorter

Now we will run the software

- `-i`: Location (and name) of the input file to analyze (expressed as a path, for example folder1/folder2/file.txt)
- `-o`: Location (and name) of the output file to write (expressed as a path, for example folder1/folder2/file.txt)
- `--nprocesses #`: Number of CPUs to use (and an integer)
- `--all`: Samples all sequences (We are warned to look at the number of sequences in our dataset since this can run into trouble with more than 10,000 reads (See your output from porechop )

In [ ]:
python3 amplicon_sorter/amplicon_sorter.py\
 -i my_edna_analysis/porechop_abi_analysis/test_sample/chop_filtered_test_sample.fastq\
 -o my_edna_analysis/amplicon_sorter_analysis/test_sample\
 --nprocesses 8\
 --similar_species 90

The results of this analysis will be a varity of [fasta](https://en.wikipedia.org/wiki/FASTA_format) files of DNA sequence at my_edna_analysis > amplicon_sorter_analysis > test_sample > choped_filtered_test_sample. 

We will take the unique sequences generates and combine them into a single file we can then use as an input for a BLAST search. 

We use the `cat` command to concatenate all the unique sequences into a file `test_sample_queries.fasta` stored at `my_edna_analysis/amplicon_sorter_analysis/test_sample/test_sample_queries.fasta`. This fille will have each grouping of potential species/barcode loci the amplicon sorter found. 


In [ ]:
cat my_edna_analysis/amplicon_sorter_analysis/test_sample/chop_filtered_test_sample/*unique.fasta > my_edna_analysis/amplicon_sorter_analysis/test_sample/test_sample_queries.fasta

## Running ampliconsorter on sample 1, 2, and 3

Create output folders for sample 1, 2, and 3

In [ ]:
mkdir -p my_edna_analysis/amplicon_sorter_analysis/XXXXX

In [ ]:
mkdir -p my_edna_analysis/amplicon_sorter_analysis/XXXXX

In [ ]:
mkdir -p my_edna_analysis/amplicon_sorter_analysis/XXXXX

Run amplicon_sorter

In [ ]:
python3 amplicon_sorter/amplicon_sorter.py\
 -i my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq\
 -o my_edna_analysis/amplicon_sorter_analysis/XXXXX\
 --nprocesses 8\
 --similar_species 90

In [ ]:
python3 amplicon_sorter/amplicon_sorter.py\
 -i my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq\
 -o my_edna_analysis/amplicon_sorter_analysis/XXXXX\
 --nprocesses 8\
 --similar_species 90

In [ ]:
python3 amplicon_sorter/amplicon_sorter.py\
 -i my_edna_analysis/porechop_abi_analysis/XXXXX/chop_filtered_XXXXX.fastq\
 -o my_edna_analysis/amplicon_sorter_analysis/XXXXX\
 --nprocesses 8\
 --similar_species 90

Concatenate unique outputs

In [ ]:
cat my_edna_analysis/amplicon_sorter_analysis/XXXXX/chop_filtered_XXXXX/*unique.fasta > my_edna_analysis/amplicon_sorter_analysis/XXXXX/XXXXX_queries.fasta

In [ ]:
cat my_edna_analysis/amplicon_sorter_analysis/XXXXX/chop_filtered_XXXXX/*unique.fasta > my_edna_analysis/amplicon_sorter_analysis/XXXXX/XXXXX_queries.fasta

In [ ]:
cat my_edna_analysis/amplicon_sorter_analysis/XXXXX/chop_filtered_XXXXX/*unique.fasta > my_edna_analysis/amplicon_sorter_analysis/XXXXX/XXXXX_queries.fasta

## BLAST search 

Finally, we can BLAST the sorted sequences to obtain BLAST results and possible identifications of our species. 

Make a folder for BLAST results for the sample

In [ ]:
mkdir -p my_edna_analysis/BLAST/test_sample

Activate BLAST software

In [ ]:
conda activate blast

Now we use `blastn` to search 

- `-query`: What we want to do searches on; a fasta file with the set of sequences output from the amplicon sorter
- `-db`: The database of known sequences to search (in this case a custom database of the rbcl and matk sequences on NCBI genbank
- `-out`: Location (and name) of the output file to write (expressed as a path, for example folder1/folder2/file.txt)
- `-evalue`: The [expectation value](https://ravilabio.info/notes/bioinformatics/e-value-bitscore.html) a measure of statistical significance for the results to return, the smaller the number, the less likely the match is due to chance. 
- `-max_target_seqs #`: The maximum number of matches to return
- `-num_threads #`: Number of CPUs to use for the search 
- `outfmt "10  qseqid stitle qcovs evalue pident"`: This complex statement (see [BLAST manual](https://www.ncbi.nlm.nih.gov/books/NBK279684/table/appendices.T.options_common_to_all_blast/)) formats our results; in this case, each blast hit returned will list the title of the match (including a GenBank accession id, match name, percentage of alignment, e-value, and percentage identity of the match. 

In [ ]:
blastn -query my_edna_analysis/amplicon_sorter_analysis/test_sample/test_sample_queries.fasta \
 -db /mnt/ceph/mar_2023_edna/e_dna_data/blast/plant_db \
 -out my_edna_analysis/BLAST/test_sample/results.txt \
 -evalue 0.01 \
 -max_target_seqs 1 \
 -num_threads 8 \
 -outfmt "10 qseqid stitle qcovs evalue pident mismatch"

Next, we can parse the BLAST results. Let's look at the first 2 results

In [ ]:
head -n2 my_edna_analysis/BLAST/test_sample/results.txt

Results are returned in a [comma delimited](https://en.wikipedia.org/wiki/Comma-separated_values) format, this means we could navigate to file `my_edna_analysis/BLAST/test_sample/results.txt` and download (left-click in the navigation pane) and open and examine this file using a program like Excel (tip, change the file extension from `.txt` to `.csv` first)

In the test sample we see (separated by commas):

- `consensus_chop_filtered_test_sample_0_0(374)`: The name of a single read record from our dataset
- `FJ493497.1 Monomastix sp. OKE-1 chloroplast`: The GenBank accession of a match followed by the name of the organism
- `complete genome`: The title of the match entry, in this case this comes from a complete genome of the matched organism, sometimes it may be just a single locus like the rbcL gene

We next have a series of numbers 

- `99`: The first number is the query coverage, if you lined up the sequences, how much our sequence would overlap (higher is better)
- `0.0`: The second number is expectation value, what is the chance this match is random; lower is better HOWEVER, this not a measure that indicates that we have a definitive match. Nearly everything we find will be a match because of common ancestry (e.g., all plants are related). 
- `90.373`: The third number is percent identity, what percentage of nucleotides do our query and match have in common (higher is better)
- `62`: The forth number is number of mismatches; how many nucleotides differ (lower is better)

We can use some Linux shell command to help us sort the data and return only the unique results. We won't explain this next command in detail but you can copy and paste it into [explain shell](https://explainshell.com/). This next command will take the results and return the unique results.

In [ ]:
cat my_edna_analysis/BLAST/test_sample/results.txt |cut -d "," -f 2|sort|uniq

This next command will take the results and return the unique results but limit to the Genus (and species name if available)

In [ ]:
cat my_edna_analysis/BLAST/test_sample/results.txt |cut -d "," -f 2|sort|uniq|cut -d " " -f 2,3|sort|uniq

### BLAST search for samples 1, 2, and 3

Make a folder for BLAST results for each sample (remember to complete the incomplete code!)


In [ ]:
mkdir -p my_edna_analysis/BLAST/

In [ ]:
mkdir -p my_edna_analysis/BLAST/

In [ ]:
mkdir -p my_edna_analysis/BLAST/

Do a blast search for each of your samples (remember to adjust these to your sample names)

In [ ]:
blastn -query my_edna_analysis/amplicon_sorter_analysis/XXXXX/XXXXX_queries.fasta \
 -db /mnt/ceph/mar_2023_edna/e_dna_data/blast/plant_db \
 -out my_edna_analysis/BLAST/XXXXX/results.txt \
 -evalue 0.01 \
 -max_target_seqs 1 \
 -num_threads 8 \
 -outfmt "10 qseqid stitle qcovs evalue pident mismatch"

In [ ]:
blastn -query my_edna_analysis/amplicon_sorter_analysis/XXXXX/XXXXX_queries.fasta \
 -db /mnt/ceph/mar_2023_edna/e_dna_data/blast/plant_db \
 -out my_edna_analysis/BLAST/XXXXX/results.txt \
 -evalue 0.01 \
 -max_target_seqs 1 \
 -num_threads 8 \
 -outfmt "10 qseqid stitle qcovs evalue pident mismatch"

In [ ]:
blastn -query my_edna_analysis/amplicon_sorter_analysis/XXXXX/XXXXX_queries.fasta \
 -db /mnt/ceph/mar_2023_edna/e_dna_data/blast/plant_db \
 -out my_edna_analysis/BLAST/XXXXX/results.txt \
 -evalue 0.01 \
 -max_target_seqs 1 \
 -num_threads 8 \
 -outfmt "10 qseqid stitle qcovs evalue pident mismatch"

In [ ]:
For each of your samples return the list of unique hits

In [ ]:
cat my_edna_analysis/BLAST/XXXXX/results.txt |cut -d "," -f 2|sort|uniq

In [ ]:
cat my_edna_analysis/BLAST/XXXXX/results.txt |cut -d "," -f 2|sort|uniq

In [ ]:
cat my_edna_analysis/BLAST/XXXXX/results.txt |cut -d "," -f 2|sort|uniq

For each of your samples return the list of unique hits, further parsed for the genus (possible also species) names

In [ ]:
cat my_edna_analysis/BLAST/XXXXX/results.txt |cut -d "," -f 2|sort|uniq|cut -d " " -f 2,3|sort|uniq

In [ ]:
cat my_edna_analysis/BLAST/XXXXX/results.txt |cut -d "," -f 2|sort|uniq|cut -d " " -f 2,3|sort|uniq

In [ ]:
cat my_edna_analysis/BLAST/XXXXX/results.txt |cut -d "," -f 2|sort|uniq|cut -d " " -f 2,3|sort|uniq

**Congratulations** on your results!

You could use the next command to turn your species name into a simple text file that could be developed into a phylogenetic tree

In [ ]:
cat my_edna_analysis/BLAST/test_sample/results.txt |cut -d "," -f 2|sort|uniq|cut -d " " -f 2,3|sort|uniq > species.txt

The `species.txt` file can be uploaded to NCBI's [Taxonomy browser](https://www.ncbi.nlm.nih.gov/Taxonomy/CommonTree/wwwcmt.cgi) for further analysis. 